In [ ]:
import numpy as np

In [ ]:
label = np.load("/home/onyxia/work/detection-habitat-spontane/train_data-PLEIADES-BDTOPO-detection-972-2022/labels/700000_1638000_56.npy")

In [ ]:
label

In [ ]:
import os
labels_dir = "/home/onyxia/work/detection-habitat-spontane/train_data-PLEIADES-BDTOPO-detection-972-2022/labels/"
for file_name in os.listdir(labels_dir):
    file_path = labels_dir + file_name
    label = np.load(file_path)
    print(label)
    print(label.shape)

In [ ]:
import sys
sys.path.append("../src/")
from run_training_pipeline import download_data, prepare_data, instantiate_lightning_module, instantiate_model, intantiate_dataloader, instantiate_trainer

In [ ]:
import yaml
from yaml.loader import SafeLoader

with open("../config.yml") as f:
    config = yaml.load(f, Loader=SafeLoader)
config

In [ ]:
model = instantiate_model(config)
model

In [ ]:
from pathlib import Path

list_path_images = []
list_path_labels = []
for name in os.listdir("/home/onyxia/work/detection-habitat-spontane/train_data-PLEIADES-BDTOPO-detection-972-2022/images/"):
    npy_name = Path(name).stem + ".npy"
    if os.path.exists("/home/onyxia/work/detection-habitat-spontane/train_data-PLEIADES-BDTOPO-detection-972-2022/labels/" + npy_name):
        list_path_images.append("/home/onyxia/work/detection-habitat-spontane/train_data-PLEIADES-BDTOPO-detection-972-2022/images/" + name)
        list_path_labels.append("/home/onyxia/work/detection-habitat-spontane/train_data-PLEIADES-BDTOPO-detection-972-2022/labels/" + npy_name)

list_path_images[:10]

In [ ]:
import shutil

list_path_images = []
list_path_labels = []
for name in os.listdir("/home/onyxia/work/detection-habitat-spontane/train_data-PLEIADES-BDTOPO-detection-972-2022/images/"):
    npy_name = Path(name).stem + ".npy"
    if os.path.exists("/home/onyxia/work/detection-habitat-spontane/train_data-PLEIADES-BDTOPO-detection-972-2022/labels/" + npy_name):
        shutil.copyfile(
            "/home/onyxia/work/detection-habitat-spontane/train_data-PLEIADES-BDTOPO-detection-972-2022/images/" + name,
            "/home/onyxia/work/detection-habitat-spontane/train_data_notebook/images/" + name
        )
        shutil.copyfile(
            "/home/onyxia/work/detection-habitat-spontane/train_data-PLEIADES-BDTOPO-detection-972-2022/labels/" + npy_name,
            "/home/onyxia/work/detection-habitat-spontane/train_data_notebook/labels/" + npy_name
        )
        list_path_images.append("/home/onyxia/work/detection-habitat-spontane/train_data_notebook/images/" + name)
        list_path_labels.append("/home/onyxia/work/detection-habitat-spontane/train_data_notebook/labels/" + npy_name)

list_path_images[:10]

In [ ]:
from run_training_pipeline import intantiate_dataset

full_dataset = intantiate_dataset(
    config, list_path_images, list_path_labels
)

In [ ]:
len(full_dataset)

In [ ]:
import torch

sample_idx = torch.randint(len(full_dataset), size=(1,)).item()
img, label, metadata = full_dataset[sample_idx]

In [ ]:
img.shape

In [ ]:
model.eval()
model(img.unsqueeze(0))

In [ ]:
img.shape

In [ ]:
label.shape

In [ ]:
label

In [ ]:
model.model

In [ ]:
model.train()

targets = []
imgs = []

target = {}
target["boxes"] = label
target["labels"] = torch.ones(len(target["boxes"])).long()
targets.append(target)
imgs.append(img)

In [ ]:
targets

In [ ]:
loss_dict = model.model(imgs, targets)

In [ ]:
loss_dict

In [ ]:
# fasterrcnn takes both images and targets for training, returns
loss = sum(loss for loss in loss_dict.values())
{"loss": loss, "log": loss_dict}

In [ ]:
list_output_dir = ["/home/onyxia/work/detection-habitat-spontane/train_data_notebook"]
train_dl, valid_dl, test_dl = intantiate_dataloader(
    config, list_output_dir
)

In [ ]:
train_dl

In [ ]:
a = next(iter(train_dl))

In [ ]:
len(a)

In [ ]:
a[0].shape

In [ ]:
a[1]

In [ ]:
a[2]

Sanity checks

In [ ]:
import random
img, bboxes, metadata = full_dataset[random.randint(0, len(full_dataset))]

In [ ]:
img = img.numpy()

In [ ]:
bboxes

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(np.transpose(img.astype(int), (1, 2, 0)))
plt.show()

In [ ]:
plt.imshow(np.transpose(img / 255., (1, 2, 0)))
plt.show()

In [ ]:
img.shape

In [ ]:
img.astype(np.uint8)

In [ ]:
img.astype(np.uint8).dtype

In [ ]:
from PIL import Image
image = Image.fromarray(np.transpose(img.astype(np.uint8), (1, 2, 0)), mode="RGB")
image

In [ ]:
from PIL import ImageFont, ImageDraw, ImageEnhance

for (x, y, xx, yy) in bboxes:
    c1 = (int(x.item()), int(y.item()))
    c2 = (int(xx.item()), int(yy.item()))
    draw = ImageDraw.Draw(image)
    draw.rectangle((c1, c2))

image

Train

In [ ]:
model = instantiate_model(config)

In [ ]:
model

In [ ]:
light_module = instantiate_lightning_module(config, model)
light_module

In [ ]:
trainer = instantiate_trainer(config, light_module)
trainer

In [ ]:
trainer.fit(light_module, train_dl, valid_dl)

In [ ]:
from classes.data.satellite_image import SatelliteImage

satellite_image = SatelliteImage(
    array=img,
    crs=0,
    bounds=0,
    transform=0,
    n_bands=3,
    filename=0,  # a adapter avec bb
    dep=0,
    date=0,
)
satellite_image

In [ ]:
bboxes

In [ ]:
from classes.data.labeled_satellite_image import DetectionLabeledSatelliteImage
labeled_image = DetectionLabeledSatelliteImage(
    satellite_image,
    label=bboxes,
    source="BDTOPO",
    labeling_date=0
)
labeled_image

In [ ]:
a = labeled_image.plot(bands_indices=(0, 1, 2))

In [ ]:
bboxes

In [ ]:
a